In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr

In [3]:
from emg_from_lfp import compute_emg
from ecephys.sglx_utils import load_trigger
import ecephys_analyses as ea

In [4]:
EMG_OPTS = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')

In [5]:
def get_emg(sig, **emg_kwargs):
    values = compute_emg(sig.values.T, sig.fs, **emg_kwargs).flatten()
    time = np.linspace(sig.time.min(), sig.time.max(), values.size)
    timedelta = pd.to_timedelta(time, "s")
    datetime = sig.datetime.values.min() + timedelta

    emg = xr.DataArray(
        values,
        dims="time",
        coords={
            "time": time,
            "timedelta": ("time", timedelta),
            "datetime": ("time", datetime),
        },
        attrs={"long_name": "emg_from_lfp", "units": "zero-lag correlation"},
    )
    for key in emg_kwargs:
        emg.attrs[key] = emg_kwargs[key]

    return emg

In [6]:
def get_condition_emg(subject, experiment, condition, probe):
    emg_chans = ea.get_channels(subject, experiment, probe, "emg_from_lfp")
    
    bin_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="lf.bin")
    emg_paths = ea.get_sglx_style_datapaths(subject=subject, experiment=experiment, condition=condition, ext="emg.nc")

    for bin_path, emg_path in zip(bin_paths, emg_paths):
        sig = load_trigger(bin_path, emg_chans)
        emg = get_emg(sig, **EMG_OPTS)
        
        # Create parent directories if they do not already exist.
        Path(emg_path).parent.mkdir(parents=True, exist_ok=True)  
        emg.to_netcdf(emg_path)
        emg.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [7]:
get_condition_emg("Doppio", "ketamine-120mg", "all", "imec1")

nChan: 385, nFileSamp: 9000060
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.016225626741, LFP nchans = 3




Done!
19:00:59: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/4-24-2020/4-24-2020_g0/4-24-2020_g0_imec1/4-24-2020_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 9000059
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.016225626741, LFP nchans = 3




Done!
19:05:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/4-24-2020/4-24-2020_g0/4-24-2020_g0_imec1/4-24-2020_g0_t1.imec1.lf.bin
nChan: 385, nFileSamp: 9000059
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.016225626741, LFP nchans = 3




Done!
19:12:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/4-24-2020/4-24-2020_g0/4-24-2020_g0_imec1/4-24-2020_g0_t2.imec1.lf.bin
nChan: 385, nFileSamp: 9000060
Filtering LFP with wp=[300, 600], ws=[275, 625], gpass=1, gstop=60,filter type=butter
Computing EMG from filtered LFP...
target sf = 20, window size = 25.0, LFP sf=2500.016225626741, LFP nchans = 3




Done!
19:17:15: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/4-24-2020/4-24-2020_g0/4-24-2020_g0_imec1/4-24-2020_g0_t3.imec1.lf.bin
